In [20]:
import numpy as np
import pandas as pd
import time

d = pd.read_csv("temp.csv")
d.head()

,KIR2D3D,GzmA,GzmB,CD8a,NKG2A,CD56,TCRab,CD4,CD49a,CD103,CD16
0,0.347520,2.248825,3.192268,0.550326,1.182181,1.557233,0.834536,0.981103,0.674518,1.141513,1.780492
1,1.354591,1.402431,0.994343,0.671340,1.194370,0.834852,0.975920,2.413024,1.475009,1.071760,0.932460
2,1.162129,0.797331,1.129649,0.595469,1.432608,0.673240,2.251479,2.310711,0.716634,0.848580,0.835219
3,0.505488,1.009671,0.931337,0.680922,1.166595,0.905019,1.698317,2.387158,1.668309,1.171005,0.752149
4,1.477766,0.772016,1.073770,0.458368,1.374335,0.795902,2.098206,2.235695,0.608443,0.855407,0.731284


In [21]:
import scanpy as sc
import anndata
import igraph
d = d.loc[:,:'CD16']
adata = anndata.AnnData(X=pd.DataFrame.to_numpy(d), obs=list(d.index.values), var=list(d.columns.values))
adata

AnnData object with n_obs × n_vars = 5653 × 11 
    obs: 0
    var: 0

In [28]:
print("Nearest neighbours...")
start = time.time()
sc.pp.neighbors(adata, n_neighbors=20)
print("DONE: {} s".format(time.time()-start))
print("Louvain clustering...")
mid = time.time()
sc.tl.louvain(adata, resolution=0.2)
print("DONE: {} s".format(time.time()-mid))
print("Number of clusters: ",max(np.array(adata.obs['louvain']).astype(int)+1))
d['cluster']=np.array(adata.obs['louvain']).astype(int)+1
d.to_csv("temp.csv", index=False)

Nearest neighbours...
DONE: 1.4652667045593262 s
Louvain clustering...
DONE: 1.5111379623413086 s
Number of clusters:  5
